In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import astropy.coordinates as coord
import astropy.units as u

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC #???????????????//
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, kendalltau

from astropy.table import QTable, join
from astropy.io import ascii
from itertools import permutations, product
from time import time

In [ ]:
# select most recent training data
train_file = 'training_data_0802.pk' # training data with 3 classes
train_file = 'training_data_1702.pk' # training data with only gcs and galaxies
#train_file = 'training_data_1902_with_stars.pk' # training data with gcs galaxies and stars, classed as 'gc' and 'non-gc'
train_file = 'training_data_2702_j.pk'
train_file = 'training_data_0203_jhk.pk'

with open(f'./pickle/training_data/{train_file}','rb') as f:
    training_data = pickle.load(f)

In [ ]:
bounds = pickle.load(open('./pickle/bounds.pk','rb'))
master_cat = pd.read_csv('./catdata/master_catalog_jan_2023.csv') #2184 objects long
cat_files = ['cat1_50.pk','cat51_100.pk','cat101_150.pk','cat151_200.pk','cat201_235.pk',
             'cat236_257.pk','cat258_279.pk','cat280_320.pk','cat321_360.pk','cat361_406.pk']

#cat files but filtered down to only 2MASS matches
cat_files_filtered = ['cat1_50_filtered.pk','cat51_100_filtered.pk','cat101_150_filtered.pk','cat151_200_filtered.pk','cat201_235_filtered.pk',
             'cat236_257_filtered.pk','cat258_279_filtered.pk','cat280_320_filtered.pk','cat321_360_filtered.pk','cat361_406_filtered.pk']
def load_cat(field, filtered=False): # change to match-case
    bounds = [50,100,150,200,235,257,279,320,360,406]
    for b in range(len(bounds)):
        if field <= bounds[b]:
            if filtered: to_load = cat_files_filtered[b]
            else: to_load = cat_files[b]
            break
    with open(f'./pickle/{to_load}','rb') as f:
        catalogue = pickle.load(f)
    return catalogue[field] # 26/02 changed to return only the selected field